<a href="https://colab.research.google.com/github/MevrouwHelderder/Assignments/blob/main/Assignment_Shark_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The essentials

In [ ]:
# Importing the essentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

%load_ext google.colab.data_table

In [49]:
# importing the dataframe
path = "/content/drive/MyDrive/attacks.csv"
df = pd.read_csv(path, encoding="ISO-8859-1")

In [50]:
# Making a copy so to not change the original dataset while modifying the shape
attacks_shape = df.copy(deep=True)

In [51]:
attacks_shape


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preparing the basics of the dataframe

In [5]:
# # Dropping columns.
# attacks = attacks.drop(
#     columns=[
#         "Date",
#         "Year",
#         "Country",
#         "Area",
#         "Location",
#         "Name",
#         "Sex ",
#         "Time",
#         "Investigator or Source",
#         "pdf",
#         "href formula",
#         "href",
#         "Case Number.1",
#         "Case Number.2",
#         "original order",
#         "Unnamed: 22",
#         "Unnamed: 23",
#     ]
# )

In [52]:
# Test: not drop time and place and sex
# Dropping columns.
attacks_shape = attacks_shape.drop(
    columns=[
        "Location",
        "Name",
        "Time",
        "Investigator or Source",
        "pdf",
        "href formula",
        "href",
        "Case Number.1",
        "Case Number.2",
        "original order",
        "Unnamed: 22",
        "Unnamed: 23",
    ]
)

In [53]:
attacks_shape

,Case Number,Date,Year,Type,Country,Area,Activity,Sex,Age,Injury,Fatal (Y/N),Species
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,Standing,F,11,Minor injury to left thigh,N,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,Surfing,M,48,Injury to left lower leg from surfboard skeg,N,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,M,NaN,Minor injury to lower leg,N,2 m shark
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#Checking on the names of the column
list(attacks_shape)

# Renaming columns.
attacks_shape = attacks_shape.rename(
    columns={"Case Number": "Case", "Sex ": "Sex", "Fatal (Y/N)": "Outcome", "Species ": "Species"})

In [56]:
# Dropping rows that have a null value on all columns except for the column "Case".
# Having only a casenumber made them useless by default.
#
# Making a copy to prevent view vs copy issues later on 


relevant_columns = list(attacks_shape.columns[1:])
attacks_shape = attacks_shape.dropna(subset=relevant_columns, how="all").copy(deep=True)


# Helpers
Below are functions that help during one or more parts of the process.
I realise that the functions that handle the non-default missing values
are not my own: we used them in an earlier exercise.

However, they are useful and it would be dumb to pretend they are my own by changing a tiny bit.

So credit where credit is due, thanks Winc!

In [133]:
# Functions to help finding non-default missing values:

# Preparing the functions for locating and adjusting the missing values.
def print_separator(sep, num, msg):
    print("\n")
    print(sep * num)
    print(f"{msg}")
    print(sep * num)

# TACTIC A: find unique values
def look_at_unique_values(column):
    unique_values_cutoff = 50
    unique_values = column.unique()
    num_unique_values = len(unique_values)
    if num_unique_values == len(column):
        print(f"Each value in the column is unique (total: {num_unique_values})")
    elif num_unique_values < unique_values_cutoff:
        print(f"Less than {unique_values_cutoff} unique values:")
        try:
            sorted = np.sort(unique_values)
            print("Values are sorted")
            display(list(sorted))
        except:
            print("Could not sort values")
            display(list(unique_values))
    else:
        print(
            f"More than {unique_values_cutoff} unique values (total: {num_unique_values})"
        )

# TACTIC B: look at the edges
def look_at_edges(df, column_name):
    # inner function
    def show_head_and_tail(values):
        num_items_to_slice = 10
        display(list(values)[:num_items_to_slice])
        display(list(values)[-num_items_to_slice:])

    column = df[column_name]
    unique_values = column.unique()
    try:
        sorted = np.sort(unique_values)
        print("Unique values sorted, head and tail:")
        show_head_and_tail(sorted)
    except TypeError as error:
        print(f"Could not sort values: {error}")
        print("..so let's try filtering NULL values and then sorting")
        print("..there could be a black sheep in the null values")
        non_null_uniques = df.loc[~df[column_name].isnull(), column_name].unique()
        sorted = np.sort(non_null_uniques)
        show_head_and_tail(sorted)

# TACTIC C: casting to a type to see if all the values match the needed type
def cast_to_type(column, maybe_type):
    try:
        column.astype(maybe_type)
        print(f"Casting to {maybe_type} was successful")
    except ValueError as error:
        print(f"Could not cast to {maybe_type}: {error}")

# TACTIC D: display the value count of the column
def value_count(column):
    display(column.value_counts(dropna=False))

# FUNCTION TO CHECK THE DATAFRAME FOR ALL FOUR TACTICS
def find_non_default_missing_values(df, column_name, maybe_type):
    long_separator_amount = 80
    short_separator_amount = 40
    # Print the header
    print_separator(
        "*",
        long_separator_amount,
        f'Finding non default missing values for column "{column_name}"',
    )
    print(f'Column "{column_name}" has datatype: {df.dtypes[column_name]}')
    column = df[column_name]
    # A
    print_separator("-", short_separator_amount, "A: Looking at unique values")
    look_at_unique_values(column)
    # B
    print_separator("-", short_separator_amount, "B: Sorting and looking at the edges")
    look_at_edges(df, column_name)
    # C
    print_separator("-", short_separator_amount, f"C: Casting to type: {maybe_type}")
    cast_to_type(column, maybe_type)
    # D
    print_separator(
        "-",
        short_separator_amount,
        "D: Looking at frequency\nAll default-NULL values will be bunched together as NaN",
    )
    value_count(column)
    print("\n")

# Function to display the default NULL values in the column.
def display_default_null_values(df, column_name):
    nulls = df.loc[df[column_name].isnull()]
    print(f'Number of default null values in "{column_name}": {len(nulls)}')

# Easier to type
nat = np.datetime64("nat")


  

In [161]:
# Function to change values based on a non-nested dictionary
# ⚠️ Mutates df
def replace_multi(df, col, dict):
    df[col] = df[col].replace(dict)
    
# Function to replace a single values with another single value.
# ⚠️ Mutates df
def replace_one(df, col, old_value, new_value):
  df[col] = df[col].replace({old_value : new_value})

# Function to replace multiple values with one value
# ⚠️ Mutates df
def replace_multi_to_one(df, col, old_list, new_value):
  df[col] = df[col].replace(dict.fromkeys(old_list, new_value))

# Function to replace using regular expressions
# ⚠️ Mutates df
def replace_regex(df, col, old_value, new_value):
  df[col] = df[col].str.replace(old_value, new_value, regex=True)


# Cleaning up the columns
Goal: keep only the data needed.

I do apologise beforehand. I suspect on some columns I did more than needed for this assignment. 

Especially when cleaning up/sorting out the last crumbs. 

I realise they probably have no significant influance and that when time and or money are an issue they probably should have been left as the were or even thrown out, depending on the goal.
But I simply had a blast and it tought me so many new skills that I took the opportunity to learn by doing.

I might also have used one tactic on a column but another on a different column. In between working on the different columns I learned a lot and I also experimented to see what worked better.

And lastly: I realise I have written some functions that could perhaps be combined. However, for now I prefer to leave them since they show the steps I took while cleaning up.

In [127]:
# Making a copy to work on, to keep track of the changes
attacks_clean = attacks_shape.copy(deep=True)

In [128]:
# Lowercase all strings and strip whitespace and/or quotationmarks around strings
attacks_clean = attacks_clean.applymap(lambda x: x.lower() if isinstance(x, str) else x)
attacks_clean = attacks_clean.applymap(lambda x: x.strip('" ') if isinstance(x, str) else x)

In [ ]:
attacks_clean

# Column: Species

In [ ]:
# Checking how everthing looks and what stands out: 
find_non_default_missing_values(attacks_clean, 'Species', 'string')

In [129]:
# Extract the species from the string where possible

# if any for of null: change to None
# if empty after stripping whitespace: change to None
# if the word shark is there: return that plus the word before "shark"
# else: return the original plus "check: "

def tidy(x):                                
    if pd.isna(x):                                    
        return None
    elif len(x.strip()) == 0:                         
        return None
    elif "shark" in x:                                
        return re.search(r"(\S+\s*)?shark", x).group()
    else:
        return f"check: {x}"                          


attacks_clean["Species"] = attacks_clean["Species"].apply(tidy)

In [130]:
# Things we can now safely change to "unconfirmed"
# Values that mention certain words.
# Values that mention no species but mention measurements like 20"] 

no_species = [
    "invalid",
    "unidentified",
    "questionable",
    "possibly",
    "not confirmed",
    "unconfirmed",
    "doubtful",
    "captive",
    "unknown",
    "several",
    "colored",
    "large shark",
    "female shark",
    "grey shark",
    "two shark",
    "the shark",
    "from shark",
    "little shark",
    "larger shark",
    "red shark",
    "young shark",
    "for shark",
    "metre shark",
    "juvenile shark",
    "gray shark",
    "finned shark",
]

# one or more digits followed by ' or " followed by
# zero or more ] followed by shark, whitespaces optional
inches = r'\d+\s*(["\']{1,})\s*\]*\s*shark'

# string containing two or less letters or digits or -
# followed by shark, whitespaces optional
small_string = r"^[a-z0-9-]{0,2}\s*shark$"

# lb or kg or foot followed by zero or more ] followed by shark, whitespaces optional
measurements = r"(kg|lb|foot)\s*\]*\s*shark"


def tidy_more(x):
    if x is not None and (
        any(word in x for word in no_species)
        or re.search(inches, x)
        or re.search(small_string, x)
        or re.search(measurements, x)
    ):
        return "unconfirmed"
    else:
        return x


attacks_clean["Species"] = attacks_clean["Species"].apply(tidy_more)

In [131]:
# removing quotation marks and this weird little fellas that look the same but are different:  
def remove_weirdos(x):
    if x is None:
        return None
    else:
        return re.sub(r'["]+', "", x)

attacks_clean["Species"] = attacks_clean["Species"].apply(remove_weirdos)

In [132]:
# Checking all values that I previously marked as 'check'
mask_species = attacks_clean["Species"].str.contains("check", na=False)
attacks_clean[mask_species]

# Changing to the right species where possible
correct_species = [
    "blue pointer",
    "wobbegong",
    "whaler",
    "hammerhead",
    "porbeagle",
    "whitetip",
    "horn",
]

def correct_checks(x):
    if x is not None and ("check" in x):
        for word in correct_species:
            if word in x:
                return f"{word} shark"
        else:
            return "unconfirmed"
    else:
        return x

attacks_clean["Species"] = attacks_clean["Species"].apply(lambda x: correct_checks(x))


In [144]:
# Checking last crumbs to clean up:
# Replace a few different writings of the same species

replacing = {"seven-gill shark": "sevengill shark",
        "7-gill shark": "sevengill shark",
        "sevengill  shark": "sevengill shark",
        "black-tipped shark": "blacktip shark",
        "blacktip  shark": "blacktip shark",
        "sand shark": "sandshark",
        "zambesi shark": "zambezi shark"}

replace_multi(attacks_clean, "Species", replacing)

In [ ]:
# attacks_clean['Species'].value_counts().head(50)
# attacks_clean['Species'].value_counts().tail(50)

# Column: Case

In [ ]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Case", "string")

# There are some duplicate values. 
# Checking what they mean: 

# attacks_clean[attacks_clean.duplicated('Case', keep=False)]

# They might indicatie cases where more then one person was attacked. 
# The columns regarding location and time might help here if needed but since 
# there are few and they seem irrelevant to our questions I will leave them for now

# Column: Type

In [106]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Type", "string")

In [145]:
# Replace a few different values that all mean the same plus replace np.nan.
replace_type = {"boating" : "boat", "boatomg" : "boat", np.nan: None}

replace_multi(attacks_clean, "Type", replace_type)

In [ ]:
attacks_clean["Type"].value_counts().head(50)
# attacks_clean["Type"].value_counts().tail(50)

# Column: Activity

In [ ]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Activity", "string")

In [148]:
# Replacing the null values
replace_activity = {".": None, np.nan: None}
replace_multi(attacks_clean, "Activity", replace_activity)

# Boarding seems to be a substantial part. Combining different spellings of the same value
boarding = ["boogie boarding", "paddle boarding", "body-boarding", "body boarding", "paddle-boarding"]
replace_multi_to_one(attacks_clean, "Activity", boarding, "bodyboarding")

In [150]:
# Taking the top 60 values as categories and dividing the other values, where possible,
# into those categories.

top_activities = list(attacks_clean["Activity"].value_counts().head(60).index)

def activities_checks(x):
    if x is not None:
      for word in top_activities:
          if word in x:
              return word
      else:
          return x

attacks_clean["Activity"] = attacks_clean["Activity"].apply(lambda x: activities_checks(x))

In [ ]:
# attacks_clean["Activity"].value_counts().head(50)
# attacks_clean["Activity"].value_counts().tail(50)

# Column: Outcome

In [ ]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Outcome", "string")

In [153]:
# Changing values to corresponding categories
replace_outcome = {"n" : "nonfatal", "y" : "fatal", "m" : "unknown", "2017": "unknown", np.nan : None}
replace_multi(attacks_clean, "Outcome", replace_outcome)

In [ ]:
# attacks_clean["Outcome"].value_counts().head(50)
# attacks_clean["Outcome"].value_counts().tail(50)

# Column: Age

In [ ]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Age", "string")

In [155]:
# change NaN to None
replace_age= {np.nan : None, "": None}
replace_multi(attacks_clean, "Age", replace_age)

In [156]:
# convert non-null values to string
non_null = attacks_clean["Age"].notnull()
attacks_clean.loc[non_null, "Age"] = attacks_clean.loc[non_null, "Age"].astype(str)

In [157]:
# Making a new column that adds age group, based on the age

# Sorting ages by range
child = [str(i) for i in range(0, 13)]
adolescent = [str(i) for i in range(13, 18)]
adult = [str(i) for i in range(18, 130)]

# Sorting all the values that describe the age in words
child_words = ["child", "2 to 3 months", "9 months", "18 months", "2½", "both 11", "6½"]
adolescent_words = ["teen", "teens"]
adult_words = ["adult" , "middle-age", "(adult)", "20s", "30s", "40s", "50s", "elderly", "60's", "60s", "mid-20s", "mid-30s", "ca. 33"]

# Sorting into age groups, also tackling values that mention multiple ages
# by splitting into seperate ages, removing any empty strings 
# and sorting into the right group IF all ages belong to the same group 
def age_groups(age):
    if age is None:
        return age
    else:
        age_list = re.split(r'[,&\s?]|(?:\s*(?:to|or)\s*)', age) 
        age_list = list(filter(None, age_list))  
        if all(word in child for word in age_list) or age in child_words:
            return "child"
        elif all(word in adolescent for word in age_list) or age in adolescent_words:
            return "adolescent"
        elif all(word in adult for word in age_list) or age in adult_words:
            return "adult"
        else:
            return None

attacks_clean.insert(4, "Age_group", attacks_clean["Age"].apply(age_groups))

# Column: Injury

I noticed a lot of injuries were the same but with different spellings.
I sorted them into broader groups.

It could have been sorted even more but I found this to be a good medium.

In [ ]:
# Checking how everything looks and what stands out: 
# find_non_default_missing_values(attacks_clean, "Injury", "string")

In [158]:
# change NaN to None
replace_injury= {np.nan : None, "": None}
replace_multi(attacks_clean, "Injury", replace_injury)

In [162]:
# Removing some words for better sorting, stripping whitespace
removable = r"\bleft\b|\bright\b|\bminor\b|\bsevere\b|\blower\b|\bto dorsum of\b|\bsmall\b"
replace_regex(attacks_clean, "Injury", removable, "no injury")
# stripping whitespace
attacks_clean["Injury"] = attacks_clean["Injury"].str.strip()

# Correcting some words for better sorting
replace_one(attacks_clean, "Injury", "injury", "injuries")

# Replace values that contain "fatal" with just "fatal"
replace_regex(attacks_clean, "Injury", r".*\bfatal\b.*", "fatal")

# Replace values that contain "no injury" with just "no injury"
replace_regex(attacks_clean, "Injury", r".*\bno injury\b.*", "no injury")

In [163]:
# Combining laceration/lacerated/etc.

# Replace "laceration" with "lacerations"
replace_regex(attacks_clean, "Injury", r"\blaceration\b", "lacerations")
# Replace "lacerated" with "lacerations"
replace_regex(attacks_clean, "Injury", r"\blacerated\b", "lacerations")

# If "lacerations" is present: place it at the front of the string
def move_lacerations_to_front(injury):
    if injury is None:
        return injury
    else: 
      if "lacerations" in injury:
         return "lacerations " + injury.replace("lacerations", "").strip()
      else:
         return injury.strip()

# Apply the function to the "Injury_2" column
attacks_clean["Injury"] = attacks_clean["Injury"].apply(move_lacerations_to_front)

In [164]:
# Combining injured/injury/etc.

# Replace "injuries" with "injury"
replace_regex(attacks_clean, "Injury", r"\binjuries\b", "injury")
# Replace "injured" with "injury to"
replace_regex(attacks_clean, "Injury", r"\binjured\b", "injury to")

# If "injury to" is present: place it at the front of the string
def move_injury_to_front(injury):
    if injury is None:
        return injury
    else: 
      if "injury to" in injury:
         return "injury to " + injury.replace("injury to", "").strip()
      else:
         return injury.strip()

# Apply the function to the "Injury_2" column
attacks_clean["Injury"] = attacks_clean["Injury"].apply(move_injury_to_front)

In [165]:
# Remove double whitespaces
replace_regex(attacks_clean, "Injury", r"\s{2,}", " ")

In [ ]:
# Checking how is looks now: 
# attacks_clean["Injury"].value_counts().head(20)
# attacks_clean["Injury"].value_counts().tail(20)

In [169]:
# Temporary copy:
attacks_clean_temp= attacks_clean.copy(deep=True)


# Column: Sex
probably unnescessary for these questions but I'm curious about the differences

In [ ]:
# Checking how everything looks and what stands out: 
find_non_default_missing_values(attacks_clean_temp, "Sex", "string")

In [172]:
# Changing Nan and the three unspecified ones to None.
replace_sex= [np.nan, "n", "lli", "."]
replace_multi_to_one(attacks_clean_temp, "Sex", replace_sex, None)

In [ ]:
# Keep temporary just in case

# # Removing some words for better sorting, stripping whitespace
# removable = r"\bleft\b|\bright\b|\bminor\b|\bsevere\b|\blower\b|\bto dorsum of\b|\bsmall\b"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(removable, "", regex=True).str.strip()

# # Correcting some words for better sorting
# replace_one(attacks_clean, "Injury", "injury", "injuries")

# # Replace values that contain "fatal" with just "fatal"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r".*\bfatal\b.*", "fatal", regex=True)

# replace_regex(attacks_clean, "Injury", r".*\bfatal\b.*", "fatal")

# # Replace values that contain "no injury" with just "no injury"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r".*\bno injury\b.*", "no injury", regex=True)

# replace_regex(attacks_clean, "Injury", r".*\bno injury\b.*", "no injury")

# # Combining laceration/lacerated/etc.

# # Replace "laceration" with "lacerations"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r"\blaceration\b", "lacerations", regex=True)
# # Replace "lacerated" with "lacerations to"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r"\blacerated\b", "lacerations", regex=True)

# # If "lacerations" is present: place it at the front of the string
# def move_lacerations_to_front(injury):
#     if injury is None:
#         return injury
#     else: 
#       if "lacerations" in injury:
#          return "lacerations " + injury.replace("lacerations", "").strip()
#       else:
#          return injury.strip()

# # Apply the function to the "Injury_2" column
# attacks_clean["Injury"] = attacks_clean["Injury"].apply(move_lacerations_to_front)

# # Sorting injured/injury/etc.

# # Replace "injuries" with "injury"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r"\binjuries\b", "injury", regex=True)

# # Replace "injured" with "injury to"
# attacks_clean["Injury"] = attacks_clean["Injury"].str.replace(r"\binjured\b", "injury to", regex=True)

# # If "injury to" is present: place it at the front of the string
# def move_injury_to_front(injury):
#     if injury is None:
#         return injury
#     else: 
#       if "injury to" in injury:
#          return "injury to " + injury.replace("injury to", "").strip()
#       else:
#          return injury.strip()

# # Apply the function to the "Injury_2" column
# attacks_clean["Injury"] = attacks_clean["Injury"].apply(move_injury_to_front)

In [167]:
attacks_clean

,Case,Date,Year,Type,Age_group,Country,Area,Activity,Sex,Age,Injury,Outcome,Species
0,2018.06.25,25-jun-2018,2018.0,boat,adult,usa,california,paddling,f,57,no injury,nonfatal,white shark
1,2018.06.18,18-jun-2018,2018.0,unprovoked,child,usa,georgia,standing,f,11,injury to thigh,nonfatal,None
2,2018.06.09,09-jun-2018,2018.0,invalid,adult,usa,hawaii,surfing,m,48,injury to leg from surfboard skeg,nonfatal,None
3,2018.06.08,08-jun-2018,2018.0,unprovoked,None,australia,new south wales,surfing,m,None,injury to leg,nonfatal,unconfirmed
4,2018.06.04,04-jun-2018,2018.0,provoked,None,mexico,colima,diving,m,None,lacerations to leg & hand shark provoked incident,nonfatal,tiger shark
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,nd.0005,before 1903,0.0,unprovoked,None,australia,western australia,diving,m,None,fatal,fatal,None
6298,nd.0004,before 1903,0.0,unprovoked,None,australia,western australia,diving,m,None,fatal,fatal,None
6299,nd.0003,1900-1905,0.0,unprovoked,None,usa,north carolina,swimming,m,None,fatal,fatal,None
6300,nd.0002,1883-1889,0.0,unprovoked,None,panama,NaN,None,m,None,fatal,fatal,None
